In [1]:
# Check that imports for the rest of the file work.
!pip install tensorflow
!pip install tensorflow-gan
!pip install tensorflow-datasets
!pip install keras
!pip install opencv-python
import tensorflow.compat.v1 as tf
import tensorflow_gan as tfgan
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

# Allow matplotlib images to render immediately.
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)  # Disable noisy outputs.
#evaluates operations immediately without building graphs

tf.disable_v2_behavior()
tf.compat.v1.enable_resource_variables()

In [2]:
import os
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import pathlib
shirtDir = os.listdir("SixDollarImages/")

import tensorflow_datasets as tfds
import tensorflow.compat.v1 as tf
import gc
import cv2

g = tf.Graph()

# with tf.Session(graph=g) as session:
def input_fn(mode, params):

  shirtListTrain = []
  shirtListTest = []
  index = 0

  for file in shirtDir:
    im = Image.open("SixDollarImages/" + file)
    im = im.resize((112, 112))
    # im = im.resize((28, 28))
    # im = im.convert("RGB")
    im = im.convert("RGB")

    # im = cv2.imread("SixDollarImages/" + file)
    # im = cv2.resize(im, (128, 128))
    # im = cv2.cvtColor(im, cv2.COLOR_RBGA2RGB)

    im = np.array(im).reshape((112, 112, 3))
    # im = np.array(im).reshape((28, 28, 3))
    im = tf.convert_to_tensor(im, dtype=tf.uint8)

    if index < 20:
        shirtListTrain.append(im)
    else:
        # shirtListTest.append(np.asarray(im))
        shirtListTest.append(im)
    del im
    index = index + 1

  assert 'batch_size' in params
  assert 'noise_dims' in params
  bs = params['batch_size']
  nd = params['noise_dims']
  split = 'train' if mode == tf.estimator.ModeKeys.TRAIN else 'test'
  shuffle = (mode == tf.estimator.ModeKeys.TRAIN)
  just_noise = (mode == tf.estimator.ModeKeys.PREDICT)
  
  #for each tensor call tf.random_normal
  #lambda is an unnamed python function 
  # _ is used to ignore certian values, here we are ignoring the index
  noise_ds = (tf.data.Dataset.from_tensors(0).repeat()
              .map(lambda _: tf.random_normal([bs, nd])))
  
  if just_noise:
    return noise_ds
  
  #define everything as tensors
  def process_path(tensorInput):
    #{'image': <tf.Tensor 'args_0:0' shape=(28, 28, 1) dtype=uint8>, 'label': <tf.Tensor 'args_1:0' shape=() dtype=int64>}
    #Step 1
    print("Step 1")
    print(tensorInput)

    image = (tf.cast(tensorInput, tf.float32) - 127.5) / 127.5
    #Tensor("truediv:0", shape=(28, 28, 1), dtype=float32)
    #Step 2
    print("Step 2")
    print(image)

    return image

  datasetTrain = tf.data.Dataset.from_tensor_slices(shirtListTrain)
  datasetTest = tf.data.Dataset.from_tensor_slices(shirtListTest)

  if split == 'train':
    #returns a map() object (which is an iterator)
    #stores data in a temporary location instead of retrieving from source everytime
    
    images_ds = datasetTrain.map(process_path)
      
  else:
    images_ds = datasetTest.map(process_path)

  if shuffle:
    images_ds = images_ds.shuffle(
        buffer_size=5, reshuffle_each_iteration=True)
  images_ds = (images_ds.batch(bs, drop_remainder=True)
              .prefetch(tf.data.experimental.AUTOTUNE))


  # <PrefetchDataset shapes: (5, 28, 28, 1), types: tf.float32>
  # Step 3
  print("Step 3")
  print(images_ds)
  return tf.data.Dataset.zip((noise_ds, images_ds))

In [3]:
# import matplotlib.pyplot as plt
# import tensorflow_datasets as tfds
# import tensorflow_gan as tfgan
# import numpy as np

# params = {'batch_size': 100, 'noise_dims':64}
# with tf.Graph().as_default():
#   ds = input_fn(tf.estimator.ModeKeys.TRAIN, params)
#   numpy_imgs = next(iter(tfds.as_numpy(ds)))[1]
# img_grid = tfgan.eval.python_image_grid(numpy_imgs, grid_shape=(10, 10))
# plt.axis('off')
# plt.imshow(np.squeeze(img_grid))
# plt.show()

In [4]:
def _dense(inputs, units, l2_weight):
  # print("Step 4")
  # print(inputs)
  # print("units")
  # print(units)
  # print("weight")
  # print(l2_weight)
  return tf.layers.dense(
      inputs, units, None,
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

def _batch_norm(inputs, is_training):
  return tf.layers.batch_normalization(
      inputs, momentum=0.999, epsilon=0.001, training=is_training)

def _deconv2d(inputs, filters, kernel_size, stride, l2_weight):
  return tf.layers.conv2d_transpose(
      inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
      activation=tf.nn.relu, padding='same',
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

def _conv2d(inputs, filters, kernel_size, stride, l2_weight):
  # print("conv2d input")
  # print(inputs)
  return tf.layers.conv2d(
      inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
      activation=None, padding='same',
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))
    
  # print("conv2d input")
  # print(inputs)
  # return tf.keras.layers.Conv2D(
  #     inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
  #     activation=None, padding='same',
  #     kernel_initializer=tf.keras.initializers.glorot_uniform,
  #     kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
  #     bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

In [5]:
def unconditional_generator(noise, mode, weight_decay=2.5e-5):
  """Generator to produce unconditional MNIST images."""
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
 
  net = _dense(noise, 1024, weight_decay)
  net = _batch_norm(net, is_training)
  net = tf.nn.relu(net)
  
  net = _dense(net, 28 * 28 * 256, weight_decay)
  net = _batch_norm(net, is_training)
  net = tf.nn.relu(net)
  
  net = tf.reshape(net, [-1, 28, 28, 256])

  net = _deconv2d(net, 64, 4, 2, weight_decay)
  net = _deconv2d(net, 64, 4, 2, weight_decay)
  # Make sure that generator output is in the same range as `inputs`
  # ie [-1, 1].
  net = _conv2d(net, 3, 4, 1, 0.0)

  net = tf.tanh(net)

  return net

In [6]:
_leaky_relu = lambda net: tf.nn.leaky_relu(net, alpha=0.01)

def unconditional_discriminator(img, unused_conditioning, mode, weight_decay=2.5e-5):
  del unused_conditioning
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  # print("disc img")
  # print(img)
  net = _conv2d(img, 64, 4, 2, weight_decay)
  net = _leaky_relu(net)
  
  net = _conv2d(net, 128, 4, 2, weight_decay)
  net = _leaky_relu(net)
  
  net = tf.layers.flatten(net)
  
  net = _dense(net, 256, weight_decay)
  net = _batch_norm(net, is_training)
  net = _leaky_relu(net)
  
  net = _dense(net, 1, weight_decay)

  return net

In [7]:
# calculate inception score with Keras
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

import tensorflow_hub as tfhub

INCEPTION_MODULE = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

def calculate_inception_score(images, n_split=1, eps=1E-16):
    processed = preprocess_input(images)

    inception_classifier_fn = tfhub.load(INCEPTION_MODULE)
    score = tfgan.eval.classifier_score(processed, inception_classifier_fn, 1)
    score.shape.assert_is_compatible_with([])
    
    return score

def calculate_frechet_distance(real_images, generated_images, num_batches=1):
   
    inception_classifier_fn = tfhub.load(INCEPTION_MODULE)
    frechet_distance = tfgan.eval.frechet_classifier_distance(real_images, generated_images, inception_classifier_fn, num_batches)
    frechet_distance.shape.assert_is_compatible_with([])

    return frechet_distance


In [8]:
from tensorflow_gan.examples.mnist import util as eval_util
import os

def get_eval_metric_ops_fn(gan_model):
  real_data_logits = tf.reduce_mean(gan_model.discriminator_real_outputs)
  gen_data_logits = tf.reduce_mean(gan_model.discriminator_gen_outputs)
  
  real_mnist_score = calculate_inception_score(gan_model.real_data)
  generated_mnist_score = calculate_inception_score(gan_model.generated_data)
  
  frechet_distance = calculate_frechet_distance(
      gan_model.real_data, gan_model.generated_data)

  fullObj =  {
      'real_data_logits': tf.metrics.mean(real_data_logits),
      'gen_data_logits': tf.metrics.mean(gen_data_logits),
      'real_mnist_score': tf.metrics.mean(real_mnist_score),
      'mnist_score': tf.metrics.mean(generated_mnist_score),
      'frechet_distance': tf.metrics.mean(frechet_distance),
  }
  print(fullObj)
  return {
      'real_data_logits': tf.metrics.mean(real_data_logits),
      'gen_data_logits': tf.metrics.mean(gen_data_logits),
      'real_mnist_score': tf.metrics.mean(real_mnist_score),
      'mnist_score': tf.metrics.mean(generated_mnist_score),
      'frechet_distance': tf.metrics.mean(frechet_distance),
  }

In [9]:
train_batch_size = 5 #@param
noise_dimensions = 64 #@param
generator_lr = 0.001 #@param
discriminator_lr = 0.0002 #@param

def gen_opt():
  gstep = tf.train.get_or_create_global_step()
  base_lr = generator_lr
  # Halve the learning rate at 1000 steps.
  lr = tf.cond(gstep < 1000, lambda: base_lr, lambda: base_lr / 2.0)
  return tf.train.AdamOptimizer(lr, 0.5)

gan_estimator = tfgan.estimator.GANEstimator(
    generator_fn=unconditional_generator,
    discriminator_fn=unconditional_discriminator,
    generator_loss_fn=tfgan.losses.wasserstein_generator_loss,
    discriminator_loss_fn=tfgan.losses.wasserstein_discriminator_loss,
    params={'batch_size': train_batch_size, 'noise_dims': noise_dimensions},
    generator_optimizer=gen_opt,
    discriminator_optimizer=tf.train.AdamOptimizer(discriminator_lr, 0.5),
    get_eval_metric_ops_fn=get_eval_metric_ops_fn)

In [10]:
# Disable noisy output.
tf.autograph.set_verbosity(0, False)

import time
steps_per_eval = 500 #@param
max_train_steps = 5000 #@param
batches_for_eval_metrics = 5 #@param

# Used to track metrics.
steps = []
real_logits, fake_logits = [], []
real_mnist_scores, mnist_scores, frechet_distances = [], [], []

cur_step = 0
start_time = time.time()

print("executing eargerly")
print(tf.executing_eagerly())

while cur_step < max_train_steps:
  next_step = min(cur_step + steps_per_eval, max_train_steps)

  start = time.time()
  print("train")
  gan_estimator.train(input_fn, max_steps=next_step)
  steps_taken = next_step - cur_step
  time_taken = time.time() - start
  print('Time since start: %.2f min' % ((time.time() - start_time) / 60.0))
  print('Trained from step %i to %i in %.2f steps / sec' % (
      cur_step, next_step, steps_taken / time_taken))
  cur_step = next_step
  
  # Calculate some metrics.
  print("evaluate")
  metrics = gan_estimator.evaluate(input_fn, steps=batches_for_eval_metrics)
  steps.append(cur_step)
  real_logits.append(metrics['real_data_logits'])
  fake_logits.append(metrics['gen_data_logits'])
  real_mnist_scores.append(metrics['real_mnist_score'])
  mnist_scores.append(metrics['mnist_score'])
  frechet_distances.append(metrics['frechet_distance'])
  print('Average discriminator output on Real: %.2f  Fake: %.2f' % (
      real_logits[-1], fake_logits[-1]))
  print('Inception Score: %.2f / %.2f  Frechet Distance: %.2f' % (
      mnist_scores[-1], real_mnist_scores[-1], frechet_distances[-1]))
  
  # Vizualize some images.
  print("predict")
  iterator = gan_estimator.predict(
      input_fn, hooks=[tf.train.StopAtStepHook(num_steps=21)])
  try:
    imgs = np.array([next(iterator) for _ in range(20)])
  except StopIteration:
    pass
  tiled = tfgan.eval.python_image_grid(imgs, grid_shape=(2, 10))
  plt.axis('off')
  plt.imshow(np.squeeze(tiled))
  plt.show()
  
  
# Plot the metrics vs step.
plt.title('MNIST Frechet distance per step')
plt.plot(steps, frechet_distances)
plt.figure()
plt.title('MNIST Score per step')
plt.plot(steps, mnist_scores)
plt.plot(steps, real_mnist_scores)
plt.show()

executing eargerly
False
train
Step 1
Tensor("args_0:0", shape=(112, 112, 3), dtype=uint8)
Step 2
Tensor("truediv:0", shape=(112, 112, 3), dtype=float32)
Step 3
<DatasetV1Adapter shapes: (5, 112, 112, 3), types: tf.float32>
C:\Users\Brad\anaconda3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
C:\Users\Brad\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\Brad\anaconda3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\normalization.py:307: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.ke

NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key metrics/InceptionV3/Conv2d_1a_3x3/BatchNorm/beta not found in checkpoint
	 [[node save/RestoreV2 (defined at C:\Users\Brad\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\estimator.py:1640) ]]

Original stack trace for 'save/RestoreV2':
  File "C:\Users\Brad\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Brad\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Brad\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Brad\anaconda3\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
    app.start()
  File "C:\Use